# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
J. Li  ->  J. Li  |  ['J. Li']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Arxiv has 71 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.13453


extracting tarball to tmp_2309.13453...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure MassFIREFLYVSTotMass.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[width=0.35\textwidth]{MassFIREFLYVSTotMass.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 85 bibliographic references in tmp_2309.14257/main.bbl.
syntax error in line 137: '=' expected
Retrieving document from  https://arxiv.org/e-print/2309.14294


extracting tarball to tmp_2309.14294...

 done.


/tmp/ipykernel_2134/3009462391.py:49: LatexWarning: 2309.14294 did not run properly
bad escape \i at position 36
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.14257-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.14257) | **iMaNGA: mock MaNGA galaxies based on IllustrisTNG and MaStar SSPs. --  III. Stellar metallicity drivers in MaNGA and TNG50**  |
|| L. Nanni, et al. -- incl., <mark>J. Neumann</mark> |
|*Appeared on*| *2023-09-26*|
|*Comments*| **|
|**Abstract**| The iMaNGA project uses a forward-modelling approach to compare the predictions of cosmological simulations with observations from SDSS-IV/MaNGA. We investigate the dependency of age and metallicity radial gradients on galaxy morphology, stellar mass, stellar surface mass density ($\Sigma_*$), and environment. The key of our analysis is that observational biases affecting the interpretation of MaNGA data are emulated in the theoretical iMaNGA sample. The simulations reproduce the observed global stellar population scaling relations with positive correlations between galaxy mass and age/metallicity quite well and also produce younger stellar populations in late-type in agreement with observations. We do find interesting discrepancies, though, that can inform the physics and further development of the simulations. Ages of spiral galaxies and low-mass ellipticals are overestimated by about 2-4 Gyr. Radial metallicity gradients are steeper in iMaNGA than in MaNGA, a discrepancy most prominent in spiral and lenticular galaxies. Also, the observed steepening of metallicity gradients with increasing galaxy mass is not well matched by the simulations. We find that the theoretical radial profiles of surface mass density $\Sigma_*$ are steeper than in observations except for the most massive galaxies. In both MaNGA and iMaNGA [Z/H] correlates with $\Sigma_*$, however, the simulations systematically predict lower [Z/H] by almost a factor of 2 at any $\Sigma_*$. Most interestingly, for galaxies with stellar mass $\log M_*\leq 10.80 M_\odot$ the MaNGA data reveal a positive correlation between galaxy radius and [Z/H] at fixed $\Sigma_*$, which is not recovered in iMaNGA. Finally, the dependence on environmental density is negligible in both the theoretical iMaNGA and the observed MaNGA data. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.14294-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.14294) | **AspGap: Augmented Stellar Parameters and Abundances for 23 million RGB  stars from Gaia XP low-resolution spectra**  |
|| <mark>J. Li</mark>, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2023-09-26*|
|*Comments*| *29 pages, 19 figures, submitted to ApJS*|
|**Abstract**| We present AspGap, a new approach to infer stellar labels from low-resolution Gaia XP spectra, including precise [$\alpha$/M] estimates for the first time. AspGap is a neural-network based regression model trained on APOGEE spectra. In the training step, AspGap learns to use XP spectra not only to predict stellar labels but also the high-resolution APOGEE spectra that lead to the same stellar labels. The inclusion of this last model component -- dubbed the hallucinator -- creates a more physically motivated mapping and significantly improves the prediction of stellar labels in the validation, particularly of [$\alpha$/M]. For giant stars, we find cross-validated rms accuracies for Teff, log g, [M/H], [$\alpha$/M] of ~1%, 0.12 dex, 0.07 dex, 0.03 dex, respectively. We also validate our labels through comparison with external datasets and through a range of astrophysical tests that demonstrate that we are indeed determining [$\alpha$/M] from the XP spectra, rather than just inferring it indirectly from correlations with other labels. We publicly release the AspGap codebase, along with our stellar parameter catalog for all giants observed by Gaia XP. AspGap enables new insights into the formation and chemo-dynamics of our Galaxy by providing precise [$\alpha$/M] estimates for 23 million giant stars, including 12 million with radial velocities from Gaia. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 36</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.13453-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.13453) | **Dissecting the Gaia HR diagram II. The vertical structure of the star  formation history across the Solar Cylinder**  |
|| A. Mazzi, et al. -- incl., <mark>M. Fouesneau</mark> |
|*Appeared on*| *2023-09-26*|
|*Comments*| *Accepted for publication in MNRAS*|
|**Abstract**| Starting from the Gaia DR3 HR diagram, we derive the star formation history (SFH) as a function of distance from the Galactic Plane within a cylinder centred on the Sun with a 200~pc radius and spanning 1.3~kpc above and below the Galaxy's midplane. We quantify both the concentration of the more recent star formation in the Galactic Plane, and the age-related increase in the scale height of the Galactic Disc stellar component, which is well-described by power-laws with indices ranging from $1/2$ to $2/3$. The vertically-integrated star formation rate falls from $(1.147 \pm 0.039)\times10^{-8}\, \text{M}_{\odot} \text{yr}^{-1} \text{pc}^{-2}$ at earlier times down to $(6.2 \pm 3.0) \times10^{-9}\, \text{M}_{\odot} \text{yr}^{-1} \text{pc}^{-2}$ at present times, but we find a significant peak of star formation in the 2 to 3 Gyr age bin. The total mass of stars formed per unit area over time is $118.7 \pm 6.2\, \text{M}_{\odot} \text{pc}^{-2}$, which is nearly twice the present stellar mass derived from kinematics within 1~kpc from the Galactic Plane, implying a high degree of matter recycling in successive generations of stars. The method is then modified by adopting an age-dependent correlation between the SFH across the different slices, which results in less noisy and more symmetrical results without significantly changing the previously mentioned quantities. This appears to be a promising way to improve SFH recovery in external galaxies. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.14257.md
    + _build/html/tmp_2309.14257/./Fig6_ZR_scatter_fit_LW.png
    + _build/html/tmp_2309.14257/./MZR_LW.png
    + _build/html/tmp_2309.14257/./MZR_MW.png
    + _build/html/tmp_2309.14257/./Fig11_sbr_morph.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# iMaNGA: mock MaNGA galaxies based on IllustrisTNG and MaStar SSPs. - III. Stellar metallicity drivers in MaNGA and TNG50

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.14257-b31b1b.svg)](https://arxiv.org/abs/2309.14257)<mark>Appeared on: 2023-09-26</mark> - 

</div>
<div id="authors">

L. Nanni, et al. -- incl., <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** The iMaNGA project uses a forward-modelling approach to compare the predictions of cosmological simulations with observations from SDSS-IV/MaNGA. We investigate the dependency of age and metallicity radial gradients on galaxy morphology, stellar mass, stellar surface mass density ( $\Sigma_*$ ), and environment. The key of our analysis is that observational biases affecting the interpretation of MaNGA data are emulated in the theoretical iMaNGA sample. The simulations reproduce the observed global stellar population scaling relations with positive correlations between galaxy mass and age/metallicity quite well and also produce younger stellar populations in late-type in agreement with observations. We do find interesting discrepancies, though, that can inform the physics and further development of the simulations. Ages of spiral galaxies and low-mass ellipticals are overestimated by about 2-4 Gyr. Radial metallicity gradients are steeper in iMaNGA than in MaNGA, a discrepancy most prominent in spiral and lenticular galaxies. Also, the observed steepening of metallicity gradients with increasing galaxy mass is not well matched by the simulations. We find that the theoretical radial profiles of surface mass density $\Sigma_*$ are steeper than in observations except for the most massive galaxies. In both MaNGA and iMaNGA [ Z/H ] correlates with $\Sigma_*$ , however, the simulations systematically predict lower [ Z/H ] by almost a factor of 2 at any $\Sigma_*$ . Most interestingly, for galaxies with stellar mass $\log M_*\leq 10.80 M_\odot$ the MaNGA data reveal a $* positive correlation*$ between galaxy radius and [ Z/H ] at fixed $\Sigma_*$ , which is not recovered in iMaNGA. Finally, the dependence on environmental density is negligible in both the theoretical iMaNGA and the observed MaNGA data.

</div>

<div id="div_fig1">

<img src="tmp_2309.14257/./Fig6_ZR_scatter_fit_LW.png" alt="Fig9" width="100%"/>

**Figure 9. -** Radial metallicity profiles, dividing the iMaNGA and MaNGA sample in stellar mass (columns) and morphology (rows) bins; see Table \ref{tab:mass}. [Z/H] is recovered with FIREFLY for both samples in the same manner. In each panel, we show the median [Z/H] in 0.1 R$_{\rm eff}$ width bins for iMaNGA (pink diamonds) and MaNGA (orange circles), considering all spaxels up to 2.5 $R_{\rm eff}$. The error bars represent the standard error on the median, see Eq. \ref{eq:standarderroronthemedian}. Linear regressions are presented up to 2.5 R$_{\rm reff}$, computed on data up to 1.5 R$_{\rm reff}$(solid violet line for iMaNGA, orange dashed lines for MaNGA).  Gradients are reported in the top-left corner of each panel for both catalogues. In the background of each panel, we show the distribution of the galaxies in the iMaNGA sample, calculated with a Gaussian kernel density estimator. (*fig:zr*)

</div>
<div id="div_fig2">

<img src="tmp_2309.14257/./MZR_LW.png" alt="Fig8.1" width="50%"/><img src="tmp_2309.14257/./MZR_MW.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** The global stellar mass-age (top panels) and -metallicity relation (bottom panels) separated by morphology, for the MaNGA and iMaNGA samples. The left panels show light-weighted quantities, and the right panels show mass-weighted ones. Age and metallicity are averaged within a central area of 3$"$ diameter. The scatter points display each galaxy in the iMaNGA sample and the line plot shows the median age and metallicity across mass bins of 0.25 dex width, for the iMaNGA (solid diamonds) and MaNGA (empty circles) galaxies. The E+S0 galaxies are represented in red, while the LTGs are represented in blue. The error bars illustrate the standard deviation. (*fig:MZR*)

</div>
<div id="div_fig3">

<img src="tmp_2309.14257/./Fig11_sbr_morph.png" alt="Fig11" width="100%"/>

**Figure 11. -** $\Sigma_*$-radius trends in the MaNGA and iMaNGA sample, in the morphology-stellar mass plane (see Table \ref{tab:mass}). $\Sigma_*$ is computed from the stellar mass recovered with FIREFLY and considering the inclination from the T-morphology for both samples, using the FIREFLY VAC dataset (see N22VAC) for the MaNGA galaxies. In each panel, we show the median  $\Sigma_*$  in 0.1 R$_{\rm eff}$ width bins, for iMaNGA (pink diamonds) and MaNGA (orange circles) galaxies, up to 2.5 $R_{\rm eff}$. The error bars represent the standard error on the median, see Eq. \ref{eq:standarderroronthemedian}. The linear regressions to the data up to 1.5 R$_{\rm reff}$ are presented (solid violet line for iMaNGA, orange dotted lines for MaNGA). The gradients are reported on the top left corner of each panel for both iMaNGA and the MaNGA galaxies. In the background of each panel, we show the distribution of the galaxies in the iMaNGA sample, calculated with a Gaussian kernel density estimator. (*fig:sbr*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.14257"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

277  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
